In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ybeoteeu
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-ybeoteeu
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266458 sha256=b9671b53c09075b59c7e125b8caad4716628779e9bb645eb6f195b6d99ec4a71
  Stored in directory: /tmp/pip-ephem-wheel-cache-d6_j8iu_/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


In [ ]:
# To prevent PyTorch bug, downgrade the installed version of numpy.
pip install numpy==1.17.4

     |████████████████████████████████| 20.0MB 47.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd

class OpenDataset(torch.utils.data.Dataset):
# Class for creating dataset and importing dataset into the Datalader
# Transforms means whether or not the image is preprocessed (left/right transform, etc.)
    def __init__(self, root, df_path, transforms=None):
        self.root = root
        self.transforms = transforms
        self.df = df_path
        names = pd.read_csv(df_path)[['filename']]
        names = names.drop_duplicates()
        self.imgs = list(np.array(names['filename'].tolist()))

    def __getitem__(self, idx):
        # Load image and check image information
        img_path = os.path.join(self.root, self.imgs[idx])
        if img_path.split('.')[-1] != 'png' : img_path += '.png'
        img = Image.open(img_path).convert("RGB")
        box_list, classes = parse_one_annot(self.df, self.imgs[idx])

        # Convert to format suitable for learning(torch.tensor type)
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        labels = torch.as_tensor(classes, dtype=torch.int64)
        image_id = torch.tensor([idx])
        # area means the area corresponding to RoI
        area_list = [(i[2] - i[0]) * (i[3] - i[1]) for i in box_list]
        areas = torch.as_tensor(area_list, dtype=torch.float32)
        # whether the roi is hidden from others
        # 0 if hidden, 1 if not
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = areas
        target["iscrowd"] = iscrowd
 
        if self.transforms is not None:
            img, target = self.transforms(img, target)
 
        return img, target
 
    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # Load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # Replace the classifier with a new one, that has
    # Get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
 
    return model

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at be37608 version check against PyTorch's CUDA version


In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
   transforms = []
   # Converts the image, a PIL image, into a PyTorch Tensor
   transforms.append(T.ToTensor())
   if train:
    # Transform the image left and right with 50% probability when learning
      transforms.append(T.RandomHorizontalFlip(0.5))
   return T.Compose(transforms)

In [ ]:
def parse_one_annot(filepath, filename):
    # Load image and check position and classname of RoI
    # At this time, convert classname to label(integer type).
    # The reason it starts from 1 is that 0 is set as the label of the background.
    data = pd.read_csv(filepath)
    boxes_array = data[data["filename"] == filename][["minX", "minY", "maxX", "maxY"]].values
    for i in range(len(boxes_array)) :
        minX = boxes_array[i, 0]
        minY = boxes_array[i, 1]
        maxX = boxes_array[i, 2]
        maxY = boxes_array[i, 3]
        if minX >= maxX : print(filename + " : " + str(minX)+", "+str(maxX))
        if minY >= maxY : print(filename + " : " + str(minY)+", "+str(maxY))
    classnames = data[data["filename"] == filename][["classname"]]
    classes = []
    for i in range(len(classnames)) :
        if classnames.iloc[i, 0] =='covid-19' : classes.append(1)
        elif classnames.iloc[i, 0] =='nodule' : classes.append(2)
        elif classnames.iloc[i, 0] =='cancer' : classes.append(3)
    return boxes_array, classes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Need to edit path
train_root = '/content/drive/My Drive/test/train/'
val_root = '/content/drive/My Drive/test/val/'
test_root = '/content/drive/My Drive/test/test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_train = OpenDataset(train_root,'/content/drive/My Drive/test/train.csv', transforms = get_transform(train=True))
dataset_val = OpenDataset(val_root,'/content/drive/My Drive/test/val.csv', transforms = get_transform(train=False))

# Randomly reorder images in a dataset
torch.manual_seed(1)
indices_train = torch.randperm(len(dataset_train)).tolist()
indices_val = torch.randperm(len(dataset_val)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices_train)
dataset_val = torch.utils.data.Subset(dataset_val, indices_val)

# Define Dataloader
data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=16, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=16, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

print("We have: {} examples, {} are training and {} testing".format(len(dataset_train)+len(dataset_val), len(dataset_train), len(dataset_val)))


We have: 15 examples, 12 are training and 3 testing


In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
num_classes = 4 # 3 class (number of classname) + 1 class (background)
# Proceed with GPU for learning but if GPU is not available, use CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# Move model to GPU or CPU
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# Construct a learning rate scheduler
# Learning rate scheduler decreases by 10x every 5 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size=5,
                                            gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    # Train for 1 epoch and print every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # Update learning rate
    lr_scheduler.step()
    # Evaluate on the validation data
    evaluate(model, data_loader_val, device=device)
torch.save(model.state_dict(), "model.pth")

Epoch: [0]  [0/3]  eta: 0:00:07  lr: 0.002502  loss: 0.7067 (0.7067)  loss_classifier: 0.1940 (0.1940)  loss_box_reg: 0.0806 (0.0806)  loss_objectness: 0.1585 (0.1585)  loss_rpn_box_reg: 0.2736 (0.2736)  time: 2.5526  data: 0.4445  max mem: 3281
Epoch: [0]  [2/3]  eta: 0:00:02  lr: 0.005000  loss: 0.6200 (0.5611)  loss_classifier: 0.1940 (0.1755)  loss_box_reg: 0.0806 (0.0815)  loss_objectness: 0.1320 (0.1324)  loss_rpn_box_reg: 0.1703 (0.1717)  time: 2.1466  data: 0.1544  max mem: 3281
Epoch: [0] Total time: 0:00:06 (2.1676 s / it)
creating index...
index created!
Test:  [0/3]  eta: 0:00:01  model_time: 0.2508 (0.2508)  evaluator_time: 0.0021 (0.0021)  time: 0.4285  data: 0.1737  max mem: 3281
Test:  [2/3]  eta: 0:00:00  model_time: 0.2421 (0.2437)  evaluator_time: 0.0011 (0.0013)  time: 0.3064  data: 0.0600  max mem: 3281
Test: Total time: 0:00:00 (0.3263 s / it)
Averaged stats: model_time: 0.2421 (0.2437)  evaluator_time: 0.0011 (0.0013)
Accumulating evaluation results...
DONE (t=0.

In [ ]:
from PIL import ImageDraw

def drawPrediction(img, label_boxes, prediction) :
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    # 예측 결과를 화면에 출력
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
        (label_boxes[elem][2], label_boxes[elem][3])], 
        outline ="green", width =3)
    for element in range(len(prediction[0]["boxes"])):
        boxes = prediction[0]["boxes"][element].cpu().numpy()
        score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                            decimals= 4)
        draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
        outline ="red", width =3)
        draw.text((boxes[0], boxes[1]), text = str(score))
    return image

In [ ]:
dataset_test = OpenDataset(test_root,'/content/drive/My Drive/test/test.csv', transforms = get_transform(train=False))
for i in range(len(dataset_test)) :
    img, _ = dataset_test[i]
    label_boxes = np.array(dataset_test[i][1]["boxes"])
    # put the model in evaluation mode
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
    prediction
    result = drawPrediction(img, label_boxes, prediction)

[{'boxes': tensor([[165., 209., 189., 221.]], device='cuda:0'),
  'labels': tensor([1], device='cuda:0'),
  'scores': tensor([0.8736],
  device='cuda:0')}]
